In [2]:
import pandas as pd
import numpy as np
import re
import pickle
import joblib
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# New Section

In [3]:
fake = pd.read_csv('Fake2.csv')

In [4]:
true = pd.read_csv('True1.csv')

In [5]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


*Now lets label the Fake News data set as 0 and Real News dataset as 1 as it will help us later to recognise that either the news is real/fake*

In [7]:
true['label'] = 1
fake['label'] = 0

In [8]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [9]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
news = pd.concat([true, fake],axis = 0)    # concatenating both the datasets together

In [11]:
news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [12]:
news.tail()

,title,text,subject,date,label
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [13]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

*So there is no null values so we dont need to do any droping of the data*

In [14]:
news = news.drop(['title','subject','date'],axis=1)    # Dropping the columns we dont need for training

In [15]:
news.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [16]:
news = news.sample(frac=1)   #Reshuffling the dataset, so that we dont get a biased model

In [17]:
news.head()

,text,label
261,The 2016 election cycle was rife with hilariou...,0
10369,BRUSSELS (Reuters) - Europe’s emerging migrati...,1
3830,WASHINGTON (Reuters) - A senior White House of...,1
2914,(Reuters) - President Donald Trump is consider...,1
7889,"Twenty-four years ago, Oprah Winfrey had a for...",0


In [18]:
news.tail()

,text,label
12877,The United Nations News Centre the official ...,0
19649,"You re carrying Mexican flags while chanting,...",0
5557,WASHINGTON (Reuters) - U.S. President Donald T...,1
15069,BEIRUT (Reuters) - Lebanon s president has tol...,1
21729,The Ferguson #BlackLivesMatter protesters are ...,0


In [19]:
news.reset_index(inplace=True)   # Resetting the index value

In [20]:
news.head()

,index,text,label
0,261,The 2016 election cycle was rife with hilariou...,0
1,10369,BRUSSELS (Reuters) - Europe’s emerging migrati...,1
2,3830,WASHINGTON (Reuters) - A senior White House of...,1
3,2914,(Reuters) - President Donald Trump is consider...,1
4,7889,"Twenty-four years ago, Oprah Winfrey had a for...",0


*Now we replace the datset field 'text' as it may contain various punctuations,HTML tags, digits, various characters*

In [21]:
def wordopt(text):
  text = text.lower()
  text = re.sub(r'https?://\S+|www\.\S+','',text)
  text = re.sub(r'<.*?>','',text)
  text = re.sub(r'[^\w\s]','',text)
  text = re.sub(r'\d','',text)
  text = re.sub(r'\n',' ',text)
  return text

In [22]:
news['text'] = news['text'].apply(wordopt)

In [23]:
news['text'].head()

0    the  election cycle was rife with hilarious   ...
1    brussels reuters  europes emerging migration p...
2    washington reuters  a senior white house offic...
3    reuters  president donald trump is considering...
4    twentyfour years ago oprah winfrey had a forme...
Name: text, dtype: object

In [24]:
news

,index,text,label
0,261,the election cycle was rife with hilarious ...,0
1,10369,brussels reuters europes emerging migration p...,1
2,3830,washington reuters a senior white house offic...,1
3,2914,reuters president donald trump is considering...,1
4,7889,twentyfour years ago oprah winfrey had a forme...,0
...,...,...,...
44893,12877,the united nations news centre the official ...,0
44894,19649,you re carrying mexican flags while chanting ...,0
44895,5557,washington reuters us president donald trump ...,1
44896,15069,beirut reuters lebanon s president has told f...,1


In [25]:
x = news['text']
y = news['label']

In [26]:
x_train , x_test , y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=42)

In [27]:
x_train.shape

(31428,)

In [28]:
x_test.shape

(13470,)

In [29]:
vectorizer = TfidfVectorizer()

In [30]:
xv_train = vectorizer.fit_transform(x_train)

In [31]:
xv_test = vectorizer.transform(x_test)

In [32]:
xv_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6476693 stored elements and shape (31428, 175386)>

In [33]:
xv_test

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2704553 stored elements and shape (13470, 175386)>

In [34]:
LR = LogisticRegression()

In [35]:
LR.fit(xv_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [36]:
pred_lr = LR.predict(xv_test)

In [37]:
LR.score(xv_test,y_test)

0.9896807720861173

In [38]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7047
           1       0.99      0.99      0.99      6423

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [39]:
DTC = DecisionTreeClassifier()

In [40]:
DTC.fit(xv_train,y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [41]:
pred_dtc = DTC.predict(xv_test)

In [42]:
DTC.score(xv_test,y_test)

0.9966592427616926

In [43]:
print(classification_report(y_test,pred_dtc))   # This model gives the overfitting data here

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7047
           1       1.00      1.00      1.00      6423

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [44]:
RFC = RandomForestClassifier()

In [45]:
RFC.fit(xv_train,y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [46]:
pred_rfc = RFC.predict(xv_test)

In [47]:
RFC.score(xv_test,y_test)

0.9889383815887156

In [48]:
print(classification_report(y_test,pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7047
           1       0.99      0.99      0.99      6423

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [49]:
joblib.dump(vectorizer,"vectorizer.jb")
joblib.dump(LR,"lr_model.jb")

['lr_model.jb']